# PRÀCTICA 2 - CLASSIFICACIÓ

1597487 | Manuel Arnau Fernández

1600123 | Alba Fernández Coronado

1605547 | Marina Palomar González

In [ ]:
# Carregant les llibreries necessàries
import math
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import collections as col
import scipy.stats as stats
import statsmodels.api as sm
import scipy.stats
import random as rand
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import LabelEncoder, StandardScaler, scale, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, HuberRegressor, RANSACRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import warnings
import time
from yellowbrick.target import FeatureCorrelation
import statistics as st
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

warnings.filterwarnings("ignore")


## B. CLASSIFICACIÓ NUMÈRICA

### EDA

In [ ]:
# Obrint el dataset
dataset = pd.read_csv('./weatherAUS.csv')
print(f'Mida de les dades: {dataset.shape}')
dataset.info()
dataset.head()

In [ ]:
#Target: variable objectiu
Y = dataset['RainTomorrow']

print("Tipus de valors: ",Y.nunique()) #quants tipus de valors hi ha: 2
print("Valors únics: ",Y.unique()) #quins són els valors únics: 0, 1
print(Y.value_counts()/len(Y)*100) #percentatge de cada resposta
print("% NaNs: ",Y.isnull().sum()/len(Y)*100) #percentatge NaNs
sns.catplot(x = 'RainTomorrow', data = dataset, kind = "count", palette='Set2')

In [ ]:
# Estudi de la correlació entre els atributs
correlacio = dataset.corr()
plt.figure(figsize=(12,10))
ax = sns.heatmap(correlacio, annot=True, linewidths=.5, cmap = sns.diverging_palette(20, 220, n=200))

In [ ]:
fig = plt.figure(figsize = (20,15))
ax = fig.gca()
dataset.hist(ax=ax)
plt.show()

In [ ]:
def season_replace(df):
    import datetime as dt
    month = []
    for num in df['Date']:
        date_obj = dt.datetime.strptime(num,"%Y-%m-%d")
        date_mon = date_obj.month
        month.append(date_mon)
    season_options = ['Summer', 'Summer', 'Autumn', 'Autumn', 'Autumn', 'Winter', 'Winter', 'Winter', 'Spring', 'Spring', 'Spring','Summer']
    seasons= []
    for i in month:
        seasons.append(season_options[i-1])
    n = df.columns[0]
    df.drop(n, axis = 1, inplace = True)
    df['Seasons'] = seasons
    df = df[['Seasons'] +  [col for col in df.columns if col != 'Seasons']]
    return df

In [ ]:
data = dataset.copy()
dataseason = season_replace(data)

rainfall =[dataseason['Seasons'], dataseason['Rainfall']]
headers = ['Seasons', 'Rainfall']
rain_df = pd.concat(rainfall, axis=1, keys=headers)

plt.figure(figsize=(8,4))
a = rain_df.groupby('Seasons').agg({'Rainfall':'sum'})
a.plot(kind='bar', color='pink')
plt.title('Rainfall distribution in each season')
plt.xlabel('Season')
plt.ylabel('Rainfall (in mm)')
plt.xticks(rotation=0)

### PREPROCESSING

In [ ]:
import missingno as msno
current_cmap = matplotlib.cm.get_cmap()
current_cmap.set_bad(color='red')
plt.imshow(dataset.isna(),aspect = 'auto')
plt.show()
# aquestes dues columnes mb tant de nans es poden eliminar pq no ens produeixen informació, segurament, 
# les seves mesures estan preses per una estació meteorologica està espatllada

# plotting the number of rows with entries per column
msno.bar(dataset)

In [ ]:
#Fem una copia del dataset
dataset_cp = dataset.copy()

#Eliminem les files que continguin NaNs a la variable relacionada amb puja
# dataset_cp = dataset_cp[dataset_cp['RainTomorrow'].notna()]
# dataset_cp = dataset_cp[dataset_cp['RainToday'].notna()]
# dataset_cp = dataset_cp[dataset_cp['Rainfall'].notna()]
dataset_cp = dataset_cp.dropna(axis=0,how='any',subset=["RainTomorrow", "Rainfall", "RainToday"])

# Mirem el percentatge de NanNs de cada variable
print((dataset_cp.isnull().sum()/len(dataset_cp))*100)

#eliminem les columnes amb percentatges alts de NaNs
dataset_cp = dataset_cp.drop(['Evaporation'], axis=1)
dataset_cp = dataset_cp.drop(['Sunshine'], axis=1)
dataset_cp = dataset_cp.drop(['Cloud9am'], axis=1)
dataset_cp = dataset_cp.drop(['Cloud3pm'], axis=1)

#omplim les columnes amb un percentatge mes baix amb la moda dels valors no nulls de la columna
dataset_cp['MinTemp'].fillna(dataset_cp['MinTemp'].mode()[0], inplace=True)
dataset_cp['MaxTemp'].fillna(dataset_cp['MaxTemp'].mode()[0], inplace=True)
dataset_cp['Rainfall'].fillna(dataset_cp['Rainfall'].mode()[0], inplace=True)
dataset_cp['WindGustDir'].fillna(dataset_cp['WindGustDir'].mode()[0], inplace=True)
dataset_cp['WindGustSpeed'].fillna(dataset_cp['WindGustSpeed'].mode()[0], inplace=True)
dataset_cp['WindDir9am'].fillna(dataset_cp['WindDir9am'].mode()[0], inplace=True)
dataset_cp['WindDir3pm'].fillna(dataset_cp['WindDir3pm'].mode()[0], inplace=True)
dataset_cp['WindSpeed9am'].fillna(dataset_cp['WindSpeed9am'].mode()[0], inplace=True)
dataset_cp['WindSpeed3pm'].fillna(dataset_cp['WindSpeed3pm'].mode()[0], inplace=True)
dataset_cp['Humidity9am'].fillna(dataset_cp['Humidity9am'].mode()[0], inplace=True)
dataset_cp['Humidity3pm'].fillna(dataset_cp['Humidity3pm'].mode()[0], inplace=True) 
dataset_cp['Pressure9am'].fillna(dataset_cp['Pressure9am'].mode()[0], inplace=True)
dataset_cp['Pressure3pm'].fillna(dataset_cp['Pressure3pm'].mode()[0], inplace=True)
dataset_cp['Temp9am'].fillna(dataset_cp['Temp9am'].mode()[0], inplace=True)
dataset_cp['Temp3pm'].fillna(dataset_cp['Temp3pm'].mode()[0], inplace=True)
dataset_cp['RainToday'].fillna(dataset_cp['RainToday'].mode()[0], inplace=True)

dataset_cp['Year'] = dataset_cp['Date'].str.split("-", n=1).str[0].astype("int64")
dataset_cp['Month'] = dataset_cp['Date'].str.split("-", n=2).str[1].astype("int64")
dataset_cp['Day'] = dataset_cp['Date'].str.split("-", n=3).str[2].astype("int64")
dataset_cp = dataset_cp.drop(['Date'], axis=1)

# Es transformes totes les dades a numeriques
dataset_cp = pd.concat([dataset_cp, pd.get_dummies(dataset_cp.RainToday, drop_first=True, prefix='RainToday')], axis=1)
dataset_cp = dataset_cp.drop(['RainToday'], axis=1)
dataset_cp = pd.concat([dataset_cp, pd.get_dummies(dataset_cp.RainTomorrow, drop_first=True, prefix='RainTomorrow')], axis=1)
dataset_cp = dataset_cp.drop(['RainTomorrow'], axis=1)
le = LabelEncoder()
dataset_cp['Location'] = le.fit_transform(dataset_cp['Location'])
dataset_cp['WindGustDir'] = le.fit_transform(dataset_cp['WindGustDir'])
dataset_cp['WindDir9am'] = le.fit_transform(dataset_cp['WindDir9am'])
dataset_cp['WindDir3pm'] = le.fit_transform(dataset_cp['WindDir3pm'])

dataset_cp.head()

In [ ]:
dataset_cp.info()

In [ ]:
#S'agafen tots els atributs que no siguin object
NoObj_data = dataset[dataset.select_dtypes(exclude=['object']).columns]

# Agafem la intersecció entre el dataset_cp i el dataset sense atributs objecte per tractar els outliers posteriorment
intersect = list(set(dataset_cp.columns).intersection(list(NoObj_data.columns)))
NoObj_data = dataset_cp[intersect]

NoObj_data.describe()

In [ ]:
plt.hist(dataset_cp['Rainfall'], color= 'green', edgecolor = 'black', alpha = 0.6)
plt.xlabel('Rainfall')
plt.show()
sns.distplot(dataset_cp['Rainfall'], color= 'green')

In [ ]:
#treiem la columna Rainfall perque els valors extrems no es consideren outliers
NoObj_data = NoObj_data.drop(['Rainfall'], axis = 1)

MiceImputed = NoObj_data.copy(deep=True) 
mice_imputer = IterativeImputer()
MiceImputed.iloc[:, :] = mice_imputer.fit_transform(NoObj_data)

# Detecting outliers with IQR
Q1 = MiceImputed.quantile(0.05)
Q3 = MiceImputed.quantile(0.95)
IQR = Q3 - Q1
total_outlier_num = ((NoObj_data < (Q1 - 1.5 * IQR)) | (NoObj_data > (Q3 + 1.5 * IQR))).sum()
print(total_outlier_num)

# Removing outliers from the dataset
dataset_cp = dataset_cp[~((MiceImputed < (Q1 - 1.5 * IQR)) |(MiceImputed > (Q3 + 1.5 * IQR))).any(axis=1)]
dataset_cp.info()

In [ ]:
# Correlacio dels atributs amb la variable objectiu
X = dataset_cp.drop(['RainTomorrow_Yes'], axis=1)
Y = dataset_cp['RainTomorrow_Yes']
feature_names = list(X.columns)
visualizer = FeatureCorrelation(labels = feature_names)
visualizer.fit(X, Y)
visualizer.poof()

In [ ]:
# Calculate the correlation matrix
corr = dataset_cp.corr()
corr1 = pd.DataFrame(abs(corr['RainTomorrow_Yes']),columns = ['RainTomorrow_Yes'])
nonvals = corr1.loc[corr1['RainTomorrow_Yes'] < 0.05]
print('Var correlation < 5%',nonvals)
nonvals = list(nonvals.index.values)

# We extract variables with correlation less than 5%
dataset_cp = dataset_cp.drop(columns=nonvals,axis=1)
print('Data Final',dataset_cp.shape)

In [ ]:
# Es separen X i Y del dataset
X = dataset_cp.drop(['RainTomorrow_Yes'], axis=1)
Y = dataset_cp['RainTomorrow_Yes']

In [ ]:
#Normalitzem les dades
scaler = StandardScaler()
X_scale = pd.DataFrame(scaler.fit_transform(X))
X_cols = X.columns
X_scale.columns = X_cols

In [ ]:
#es fa un pca per visualitzar els outliers
model = PCA(n_components=8)
model.fit(X_scale)
data_pca = pd.DataFrame(
    data = model.components_,
    columns = X.columns,
    index = ['PC1','PC2','PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8'])
X_pca = model.transform(X_scale)

# Percentatge de variança de cada component
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
ax.bar(
    x      = np.arange(model.n_components_) + 1,
    height = model.explained_variance_ratio_
)

for x, y in zip(np.arange(len(data_pca.columns)) + 1, model.explained_variance_ratio_):
    label = round(y, 2)
    ax.annotate(
        label,
        (x,y),
        textcoords="offset points",
        xytext=(0,10),
        ha='center'
    )

ax.set_xticks(np.arange(model.n_components_) + 1)
ax.set_ylim(0, 1.1)
ax.set_title('Percentatge de variança explicat per cada ')
ax.set_xlabel('Component principal')
ax.set_ylabel('% variança explicat');

#Conclusió: com les variances són molt baixes, no val la pena fer un PCA.

In [ ]:
# No val la pena fer un polynomial features perque tenim masses característiques i trigariem molt

### MODEL SELECTION

In [ ]:
#Separem en train i test
x_train,x_test,y_train,y_test = train_test_split(X_scale,Y,test_size=0.3, random_state = 0,shuffle=True)

In [ ]:
#FEATURE SELECTION
from statsmodels.stats.outliers_influence import variance_inflation_factor

def BWSelection_VIF(Xtrain):
    # VIF dataframe
    vif_data = pd.DataFrame()
    vif_data["feature"] = Xtrain.columns
    # calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(Xtrain.values, i)
                              for i in range(len(Xtrain.columns))]

    #si el valor de VIF és molt alt, hi ha molta correlacio entre variables
    if max(vif_data["VIF"]) > 5:
        X_new = Xtrain.drop(vif_data['feature'][np.argmax(vif_data["VIF"])],axis=1)
        BWSelection_VIF(X_new)
    else:
        llistaVIF.append([col for col in Xtrain])

In [ ]:
llistaVIF =[]
BWSelection_VIF(X_scale)
llistaVIF = llistaVIF[0]
llistaVIF

In [ ]:
#MÚLTIPLE DECISION TREE CLASSIFIER
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from joblib import Parallel, delayed

#Treballem en paral·lel per fer un model simple per a cada atribut considerat important en el VIF_Selection
#A partir d'aquí decidirem amb quins models treballar
def MultipleDT(atribut):
    
    clf_dt = DecisionTreeClassifier(random_state=0)
    clf_dt.fit(x_train[[atribut]],y_train)
    y_pred1 = clf_dt.predict(x_test[[atribut]])
    score1 = accuracy_score(y_test,y_pred1)
    y_pred2 = clf_dt.predict(x_train[[atribut]])
    score2 = accuracy_score(y_train,y_pred2)
    return [score1,score2]

In [ ]:
t0=time.time() 
result = Parallel(n_jobs = 10)(delayed(MultipleDT)(atribut) for atribut in llistaVIF)

for atribut,r in zip(llistaVIF,result): 
        print('Accuracy',atribut,':',r) #r = [acc_test, acc_train]
print('Time taken :' , time.time()-t0, "s")

print("Null accuracy: ",y_test.value_counts()[0]/sum(y_test.value_counts())) #Percentatge de 0 al y_test

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
# SVM
def MultipleSVC(k):
    model = svm.SVC(kernel = k) #classificació
    model.fit(x_train[['Humidity3pm']], y_train)
    y_pred = model.predict(x_test[['Humidity3pm']])
    score = accuracy_score(y_test,y_pred)
    return score

# LOGISTIC REGRESSION
def LogReg():
    logreg = LogisticRegression(solver='saga', random_state=0)
    logreg.fit(x_train[llistaVIF], y_train)
    score_test = logreg.score(x_test[llistaVIF], y_test)
    score_train = logreg.score(x_train[llistaVIF], y_train)
    LM_cm = confusion_matrix(y_test, logreg.predict(x_test[llistaVIF]))
    return logreg, score_test, score_train, LM_cm
    
# KNN
def KNN(n=5):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(x_train[llistaVIF],y_train)
    score_test = knn.score(x_test[llistaVIF], y_test)
    score_train = knn.score(x_train[llistaVIF], y_train)
    KNN_cm = confusion_matrix(y_test, knn.predict(x_test[llistaVIF]))
    return knn, score_test, score_train, KNN_cm
    
#Preceptron model
def Perc():
    clf = Perceptron(random_state=0)
    clf.fit(x_train, y_train)
    score_test = clf.score(x_test,y_test)
    score_train = clf.score(x_train,y_train)
    return clf, score_test, score_train

#Random Forest
def RandomForest():
    modelrfcla = RandomForestClassifier(n_estimators=50,random_state=9,n_jobs=-1).fit(x_train[llistaVIF],y_train)
    score_test = modelrfcla.score(x_test[llistaVIF], y_test)
    score_train = modelrfcla.score(x_train[llistaVIF], y_train)
    RF_cm = confusion_matrix(y_test, modelrfcla.predict(x_test[llistaVIF]))
    return modelrfcla, score_test, score_train, RF_cm

In [ ]:
# SVM
print("MODEL SVM")
t0=time.time()        
kernels = ["sigmoid", "rbf"]
result = Parallel(n_jobs = 2)(delayed(MultipleSVC)(kernel) for kernel in kernels)
for kernel,r in zip(kernels,result):
        print('Accuracy',kernel,':',r)
print('Time taken :' , time.time()-t0)

# LR
print("MODEL LOGISTIC REGRESSION")
t0=time.time()  
LR, log_acctest, log_acctrain, LR_cm = LogReg()
print("Accuracy test:",log_acctest)
print("Accuracy train:",log_acctrain)
print('Time taken :' , time.time()-t0)

# KNN
print("MODEL KNN")
t0=time.time()
KNN, knn_acctest, knn_acctrain, KNN_cm = KNN()
print("Accuracy test:",knn_acctest)
print("Accuracy train:",knn_acctrain)
print('Time taken :' , time.time()-t0)

#Perceptron
print("MODEL PERCEPTRON")
t0=time.time() 
Perc, perc_acctest, perc_acctrain = Perc()
print("Accuracy test:",perc_acctest)
print("Accuracy train:",perc_acctrain)
print('Time taken :' , time.time()-t0)

#Random Forest
print("MODEL RANDOM FOREST")
t0=time.time() 
RF, RF_acctest, RF_acctrain, RF_cm = RandomForest()
print("Accuracy test:",RF_acctest)
print("Accuracy train:",RF_acctrain)
print('Time taken :' , time.time()-t0)

### CROSSVALIDATION

In [ ]:
#ens qdem amb el logistic regression, Random forest i amb el KNN q son els millors models
from sklearn.model_selection import StratifiedKFold
def CV (k):
    skf = StratifiedKFold(n_splits=k)
    skf.get_n_splits(X, Y)
    StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    i = 1
    for train_index, test_index in skf.split(X_scale, Y):
        X_train, X_test = X_scale.iloc[train_index], X_scale.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

        #Logistic Regression
        LR.fit(X_train[llistaVIF], Y_train)
        y_pred = LR.predict(X_test[llistaVIF])
        score = accuracy_score(Y_test, y_pred)
        accLM.append(score)
#         print("Accuracy Logistic Model: ",score,"fold: ", i)

        #KNN
        KNN.fit(X_train[llistaVIF], Y_train)
        pred_test = KNN.predict(X_test[llistaVIF])
        score = accuracy_score(Y_test, pred_test)
        accKNN.append(score)
#         print("Accuracy KNN Model: ",score,"fold: ", i)

        #RandomForest
        RF.fit(X_train[llistaVIF], Y_train)
        pred_test = RF.predict(X_test[llistaVIF])
        score = accuracy_score(Y_test, pred_test)
        accRF.append(score)
#         print("Accuracy RandomForest Model: ",score,"fold: ", i)
        i = i+1
    return accLM, accKNN, accRF

#No  seria convenient aplicar LeaveOneOut, ja que en aquest dataset hi ha  moltes files i trigariem molt

In [ ]:
k = list(range(2,8))
for split in k:
    print("CrossValidation amb k = ",split)
    accLM = []
    accKNN = []
    accRF = []
    accLM, accKNN, accRF = CV(split)
    plt.plot(list(range(2,split+1)), accLM, label = "LR - Accuracy mitjà: {:.3f}".format(st.mean(accLM)))
    plt.plot(list(range(2,split+1)), accKNN, label = "KNN - Accuracy mitjà: {:.3f}".format(st.mean(accKNN)))
    plt.plot(list(range(2,split+1)), accRF, label = "RF - Accuracy mitjà: {:.3f}".format(st.mean(accRF)))
    plt.legend(loc = "lower right")
    plt.show()

### METRIC ANALYSIS

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(1, 3, 1) 
ax1.set_title('Logistic Regression Classification') 
ax2 = fig.add_subplot(1, 3, 2) 
ax2.set_title('KNN Classification')
ax3 = fig.add_subplot(1, 3, 3)
ax3.set_title('Random Forest Classification')
sns.heatmap(data=LR_cm, annot=True, linewidth=0.7, linecolor='cyan',cmap="YlGnBu" ,fmt='g', ax=ax1)
sns.heatmap(data=KNN_cm, annot=True, linewidth=0.7, linecolor='cyan',cmap="YlGnBu" ,fmt='g', ax=ax2)  
sns.heatmap(data=RF_cm, annot=True, linewidth=0.7, linecolor='cyan',cmap="YlGnBu" ,fmt='g', ax=ax3)
plt.show()

In [ ]:
from yellowbrick.classifier import ROCAUC

#Logistic Regression
visualizerLR = ROCAUC(LR)

visualizerLR.fit(x_train[llistaVIF], y_train)        # Fit the training data to the visualizer
visualizerLR.score(x_test[llistaVIF], y_test)        # Evaluate the model on the test data
visualizerLR.show()                       # Finalize and show the figure

#KNN
KNN.fit(x_train[llistaVIF], y_train)
visualizerKNN = ROCAUC(KNN)

visualizerKNN.fit(x_train[llistaVIF], y_train)        # Fit the training data to the visualizer
visualizerKNN.score(x_test[llistaVIF], y_test)        # Evaluate the model on the test data
visualizerKNN.show()                       # Finalize and show the figure


#RandomForest
RF.fit(x_train[llistaVIF], y_train)
visualizerRF = ROCAUC(RF)

visualizerRF.fit(x_train[llistaVIF], y_train)        # Fit the training data to the visualizer
visualizerRF.score(x_test[llistaVIF], y_test)        # Evaluate the model on the test data
visualizerRF.show()                       # Finalize and show the figure

In [ ]:
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import plot_confusion_matrix, f1_score, average_precision_score
# Compute Precision-Recall and plot curve LogisticRegression
def PrecisionRecall (model):
    precision = {}
    recall = {}
    average_precision = {}
    plt.figure()
    probs = model.predict_proba(x_test[llistaVIF])

    for i in range(2):
        precision[i], recall[i], _ = precision_recall_curve(y_test == i, probs[:, i])
        average_precision[i] = average_precision_score(y_test == i, probs[:, i])
        plt.plot(recall[i], precision[i],
        label='Precision-recall curve of class {0} (area = {1:0.2f})'
                               ''.format(i, average_precision[i]))
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.legend(loc="upper right")
        
    plt.title(model)
    plt.show()

In [ ]:
models = [LR, KNN, RF]
for m in models:
    PrecisionRecall(m)

In [ ]:
from sklearn.metrics import classification_report
#Logistic Regression
print("LOGISTIC REGRESSION CLASSIFICATION REPORT")
print(classification_report(y_test, LR.predict(x_test[llistaVIF])))

#KNN
print("KNN CLASSIFICATION REPORT")
print(classification_report(y_test, KNN.predict(x_test[llistaVIF])))

#Random Forest
print("RANDOM FOREST CLASSIFICATION REPORT")
print(classification_report(y_test, RF.predict(x_test[llistaVIF])))

### HYPERPARAMETER SEARCH

In [ ]:
from sklearn.model_selection import GridSearchCV

#Logistic Regression
solvers = ['newton-cg', 'lbfgs', 'liblinear','saga']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = StratifiedKFold(n_splits=10, random_state=1, shuffle = True)
grid_search = GridSearchCV(estimator=LR, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_scale[llistaVIF],Y)
# summarize results
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
#KNN
n_neighbors = [1,2,5,10]

# define grid search
grid = dict(n_neighbors = n_neighbors)
cv = StratifiedKFold(n_splits=10, random_state=1, shuffle = True)
grid_search = GridSearchCV(estimator=KNN, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_scale[llistaVIF],Y)
# summarize results
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
#RF
n_estimators = [10,25,50]
max_features = ['sqrt']
min_samples_split = [1000,500,50,10]
max_depth= [10,25,None] 
# define grid search
grid = dict(n_estimators=n_estimators,max_features=max_features,min_samples_split=min_samples_split, max_depth = max_depth)
cv = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
grid_search = GridSearchCV(estimator=RF, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_scale, Y)
# summarize results
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## A. COMPARATIVA DE MODELS

In [ ]:
from sklearn import svm, datasets

# Take the first two features.
X = X_scale[['Humidity3pm','Rainfall']]
n_classes = 2
fig, sub = plt.subplots(1, 2, figsize=(16,6))
sub[0].scatter(X[['Humidity3pm']], Y, c=Y, cmap=plt.cm.coolwarm, edgecolors='k')
sub[1].scatter(X[['Rainfall']], Y, c=Y, cmap=plt.cm.coolwarm, edgecolors='k')

particions = [0.5]

for part in particions:
    x_t, x_v, y_t, y_v = train_test_split(X, Y, train_size=part)
    
    #Creem el regresor logístic
    logireg = LogisticRegression(C=2.0, fit_intercept=True, penalty='l2', tol=0.001)

    # l'entrenem
    logireg.fit(x_t, y_t)

    print ("Correct classification Logistic ", part, "% of the data: ", logireg.score(x_v, y_v))
    
    #Creem el super vector machine
    svc = svm.SVC(C=10.0, kernel='rbf', gamma=0.9, probability=True,max_iter=10000)

    # l'entrenem 
    svc.fit(x_t, y_t)
    probs = svc.predict_proba(x_v)
    print ("Correct classification SVM      ", part, "% of the data: ", svc.score(x_v, y_v))


In [ ]:
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

def show_C_effect(C=1.0, gamma=0.7, degree=3):

    # import some data to play with
    # Take the first two features. We could avoid this by using a two-dim dataset
    X = dataset_cp[['Humidity3pm','Rainfall']]

    # we create an instance of SVM and fit out data. We do not scale our
    # data since we want to plot the support vectors
    # title for the plots
    titles = (
              'LinearSVC C=0.1','LinearSVC C=1','LinearSVC C=10','LinearSVC C=100'
              
              )

    models = (svm.LinearSVC(C=0.1,max_iter=10000),
              svm.LinearSVC(C=1, max_iter=10000),
              svm.LinearSVC(C=10, max_iter=10000),
              svm.LinearSVC(C=100, max_iter=10000))
                
            
    models = (clf.fit(X, Y) for clf in models)

    plt.close('all')
    fig, sub = plt.subplots(2, 2, figsize=(14,9))
    plt.subplots_adjust(wspace=0.4, hspace=0.4)

    X0 = np.linspace(np.min(X[['Humidity3pm']]),np.max(X[['Humidity3pm']]),100)
    X1 = np.linspace(np.min(X[['Rainfall']]),np.max(X[['Rainfall']]),100)

    xx, yy = make_meshgrid(X0, X1)
    for clf, title, ax in zip(models, titles, sub.flatten()):
        plot_contours(ax, clf, xx, yy,
                      cmap=plt.cm.coolwarm, alpha=0.8)
        ax.scatter(X['Humidity3pm'],X['Rainfall'],c=Y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xlabel('Humidity3pm')
        ax.set_ylabel('Rainfall')
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_title(title)

    plt.show()


In [ ]:
show_C_effect(C=0.1) # c molt gran, farà underfitting (posarà tots els coeficients a zero), train sempre ho farà pitjor, 
                    # allí on el train i test es trobin, c és la optima
                    # fer grafica C-J on J=-sum(ylog(1-yp)+(1-y)log(yp)+c*sum(h^2))